<a href="https://colab.research.google.com/github/marcelogarret/rna/blob/main/aula12_transformer_pipeline_garret_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Baixando e extraindo o dataset IMDB
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  10.6M      0  0:00:07  0:00:07 --:--:-- 15.4M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [2]:
# Criando diretórios base
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"

# Organizando os dados em diretórios de treino e validação
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [3]:
# Importando o pipeline da biblioteca transformers para análise de sentimentos
from transformers import pipeline

# Pipeline de análise de sentimentos
sentiment_analysis = pipeline("sentiment-analysis",  model="distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
# Testando o pipeline

# Função para ler arquivos de texto
def read_reviews(folder):
    reviews = []
    for file_name in os.listdir(folder):
        if file_name.endswith(".txt"):
            with open(os.path.join(folder, file_name), "r", encoding="utf-8") as f:
                reviews.append(f.read())
    return reviews

# Lendo alguns reviews positivos e negativos
random.seed(42)
pos_reviews = random.sample(read_reviews("aclImdb/test/pos"), 3)
neg_reviews = random.sample(read_reviews("aclImdb/test/neg"), 3)

# Analisando os sentimentos dos reviews positivos
print("Analisando reviews positivos")
for review in pos_reviews:
    result = sentiment_analysis(review[:512])  # Limite de 512 tokens por resenha
    print(f"Texto: {review[:100]}...\nSentimento: {result}\n")

# Analisando os sentimentos dos reviews negativos
print("Analisando reviews negativos")
for review in neg_reviews:
    result = sentiment_analysis(review[:512])  # Limite de 512 tokens por resenha
    print(f"Texto: {review[:100]}...\nSentimento: {result}\n")

Analisando reviews positivos
Texto: Maybe I'm reading into this too much, but I wonder how much of a hand Hongsheng had in developing th...
Sentimento: [{'label': 'NEGATIVE', 'score': 0.9944010376930237}]

Texto: I gave this a 10 because it's the best film of its kind...a good old-fashioned T & A film in the sha...
Sentimento: [{'label': 'POSITIVE', 'score': 0.9997534155845642}]

Texto: I saw 'Begotten' last night, and I'm of two minds on the film.<br /><br />On one hand, I appreciate ...
Sentimento: [{'label': 'POSITIVE', 'score': 0.8414876461029053}]

Analisando reviews negativos
Texto: If you`re not old enough to remember Yoram Globus and Menahem Golan here`s the rundown : They`re a c...
Sentimento: [{'label': 'NEGATIVE', 'score': 0.9959830045700073}]

Texto: On the way home from a day at the beach, four young people seek shelter from a torrential downpour a...
Sentimento: [{'label': 'NEGATIVE', 'score': 0.9978360533714294}]

Texto: Bugsy Siegel was 31 when he went out to the West C

In [5]:
# Avaliando o dataset de teste com o pipeline da biblioteca transformers para análise de sentimentos
from sklearn.metrics import accuracy_score

# Função para ler arquivos de texto
def read_reviews(folder):
    reviews = []
    for file_name in os.listdir(folder):
        if file_name.endswith(".txt"):
            with open(os.path.join(folder, file_name), "r", encoding="utf-8") as f:
                reviews.append(f.read())
    return reviews

# Lendo todos os reviews positivos e negativos do dataset de teste
pos_reviews = read_reviews("aclImdb/test/pos")
neg_reviews = read_reviews("aclImdb/test/neg")

# Gerando labels positivos e negativos para o dataset de teste
pos_labels = [1] * len(pos_reviews)  # 1 para reviews positivos
neg_labels = [0] * len(neg_reviews)  # 0 para reviews negativos

# Juntando todos os reviews e labels
test_reviews = pos_reviews + neg_reviews
test_labels = pos_labels + neg_labels

# Truncando os textos para um comprimento máximo de 512 tokens
test_reviews = [review[:512] for review in test_reviews]

# Analisando os sentimentos dos reviews do dataset de teste
print("Analisando reviews do dataset de teste")
predictions = []
for review in test_reviews:
  result=sentiment_analysis(review)[0]['label']
  predictions.append(1 if result == 'POSITIVE' else 0)

# Calculando a acurácia do pipeline
accuracy = accuracy_score(test_labels, predictions)
print(f"Acurácia: {accuracy:.2f}")

Analisando reviews do dataset de teste
Acurácia: 0.83
